<a href="https://colab.research.google.com/github/chottokun/spaCy_training/blob/main/spacy_llm20230513.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://github.com/explosion/spacy-llm

In [ ]:
# https://github.com/explosion/spacy-llm

In [1]:
!pip install -q transformers python-dotenv
!pip install -q spacy-llm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.0 MB/s eta 0:00:00


In [2]:
# Google Driveをマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
# 秘密情報を記載したファイルの場所を指定
envfile_mount = '/content/drive/MyDrive/Colab Notebooks/env/'
# 秘密情報が保存されているテキストファイルのパス
secrets_file_path = os.path.join(envfile_mount, 'env.txt')
# 環境変数をワークディレクトリにコピー
import shutil
shutil.copyfile(secrets_file_path, ".env")
# .env ファイルをロードして環境変数へ反映
from dotenv import load_dotenv
load_dotenv()

True

In [19]:
%%bash
cat << EOF > config.cfg
[nlp]
lang = "en"
pipeline = ["llm"]

[components]

[components.llm]
factory = "llm"

[components.llm.task]
@llm_tasks = "spacy.TextCat.v1"
labels = COMPLIMENT,INSULT

[components.llm.backend]
@llm_backends = "spacy.REST.v1"
api = "OpenAI"
config = {"model": "gpt-3.5-turbo", "temperature": 0.3}
EOF

In [20]:
from spacy import util

config = util.load_config("config.cfg")
nlp = util.load_model_from_config(config, auto_fill=True)
doc = nlp("You look gorgeous!")
print(doc.cats)

{'COMPLIMENT': 1.0, 'INSULT': 0.0}


In [21]:
doc = nlp("おまえは危険な人物に見える。")
print(doc.cats)

{'COMPLIMENT': 0.0, 'INSULT': 1.0}


In [23]:
%%bash
cat << EOF > config.cfg
[nlp]
lang = "en"
pipeline = ["llm"]

[components]

[components.llm]
factory = "llm"

[components.llm.task]
@llm_tasks = "spacy.NER.v1"
labels = PERSON,ORGANISATION,LOCATION

[components.llm.backend]
@llm_backends = "spacy.REST.v1"
api = "OpenAI"
config = {"model": "gpt-3.5-turbo", "temperature": 0.3}
EOF

In [24]:
from spacy import util

config = util.load_config("config.cfg")
nlp = util.load_model_from_config(config, auto_fill=True)
doc = nlp("Jack and Jill rode up the hill in Les Deux Alpes")
print([(ent.text, ent.label_) for ent in doc.ents])

[('Jack', 'PERSON'), ('Jill', 'PERSON'), ('Les Deux Alpes', 'LOCATION')]


In [25]:
doc = nlp("ジャックとジルは青森の丘に登った。")
print([(ent.text, ent.label_) for ent in doc.ents])

[]
